In [1]:
import numpy as np

def vec2string(SG_vecs,lattice_size,operator):
    '''
    This reverses the following encoding of SG strings to SG vectors
    
    ZZIZZIZZIIII can be written as
    
          matrix    vector
    ZZI     110       1
    ZZI ->  110   ->  1
    ZZI     110       1
    III     000       0
    
    IXXIXXIIIIII can be written as
    
          matrix    vector
    IXX     011
    IXX ->  011   -> 011
    III     000
    III     000
    
    Input: SG vectors [# of row/column of tiles]
                      [# of separate strings in each row/column]
    Output: all SG string
    
    '''
    nrow, ncol = lattice_size
    d = {0:'I',1:operator}
    all_strings = []
    for i in range(len(SG_vecs)):
        for g in SG_vecs[i]:
            g_mat = np.zeros((nrow,ncol),dtype=int)
            if len(g) == nrow:
                g_mat[:,i] = g_mat[:,i+1] = g
            else:
                g_mat[i,:] = g_mat[i+1,:] = g
            all_strings.append(''.join( np.vectorize(d.get)(g_mat).flatten() ))
    return all_strings

def cut_SG_subgroup(SG_subgroup,pos):
    '''
    Example: 2 SG vectors -> 3 SG vectors
      cut here
         |
    [[1,1,1,0],  ->  [[1,1,0,0], -- update cut generator
     [0,0,0,1]]       [0,0,0,1],
                      [0,0,1,0]] -- append new generator
    
    Input: SG subgroup (one column/row)
    Output: updated SG subgroup after cut at position
    '''
    subgroup = SG_subgroup.copy()
    cut_pos = np.argmax(subgroup[:,pos])
    g_to_cut = subgroup[cut_pos]
    g0,g1 = np.zeros_like(g_to_cut),np.zeros_like(g_to_cut)
    g0[:pos+1] = g_to_cut[:pos+1]
    g1[pos+1:] = g_to_cut[pos+1:]
    if not g1.any():
        print('---Cut is already performed!---')
        return subgroup
    subgroup[cut_pos] = g0
    subgroup = np.vstack([subgroup,g1])
    return subgroup

In [2]:
# Assuming Z stabilizers are 2-column wide and spread vertically
nrow = 4
ncol = 3
SG_vecs_Z = [np.ones([1,nrow],dtype=int)]*(ncol-1)
SG_vecs_X = [np.ones([1,ncol],dtype=int)]*(nrow-1)
print('Original stabilizer generators:')
print(vec2string(SG_vecs_Z,(nrow,ncol),'Z'))
print(vec2string(SG_vecs_X,(nrow,ncol),'X'))

# coloring
Z_tiles = [(0,0),(2,0),(2,1)]
print(f'\nZ stabilizers after coloring tiles')
for i,j in Z_tiles:
    SG_vecs_Z[j] = cut_SG_subgroup(SG_vecs_Z[j],i)
    print(f'{(i,j)}: ',vec2string(SG_vecs_Z,(nrow,ncol),'Z'))


# coloring
X_tiles = [(0,1),(1,0),(1,1)]
print(f'\nX stabilizers after coloring tiles')
for i,j in X_tiles:
    SG_vecs_X[i] = cut_SG_subgroup(SG_vecs_X[i],j) # note how i and j switch between X and Z
    print(f'{(i,j)}: ',vec2string(SG_vecs_X,(nrow,ncol),'X'))


Original stabilizer generators:
['ZZIZZIZZIZZI', 'IZZIZZIZZIZZ']
['XXXXXXIIIIII', 'IIIXXXXXXIII', 'IIIIIIXXXXXX']

Z stabilizers after coloring tiles
(0, 0):  ['ZZIIIIIIIIII', 'IIIZZIZZIZZI', 'IZZIZZIZZIZZ']
(2, 0):  ['ZZIIIIIIIIII', 'IIIZZIZZIIII', 'IIIIIIIIIZZI', 'IZZIZZIZZIZZ']
(2, 1):  ['ZZIIIIIIIIII', 'IIIZZIZZIIII', 'IIIIIIIIIZZI', 'IZZIZZIZZIII', 'IIIIIIIIIIZZ']

X stabilizers after coloring tiles
(0, 1):  ['XXIXXIIIIIII', 'IIXIIXIIIIII', 'IIIXXXXXXIII', 'IIIIIIXXXXXX']
(1, 0):  ['XXIXXIIIIIII', 'IIXIIXIIIIII', 'IIIXIIXIIIII', 'IIIIXXIXXIII', 'IIIIIIXXXXXX']
(1, 1):  ['XXIXXIIIIIII', 'IIXIIXIIIIII', 'IIIXIIXIIIII', 'IIIIXIIXIIII', 'IIIIIXIIXIII', 'IIIIIIXXXXXX']
